<img src="https://raw.githubusercontent.com/alan-barzilay/NLPortugues/master/imagens/logo_nlportugues.png"   width="150" align="right">

# Lista 4 - Word2Vec

______________

Nessa lista nós exploraremos o espaço vetorial gerado pelo algoritmo Word2Vec e algumas de suas propriedades mais interessantes. Veremos como palavras similares se organizam nesse espaço e as relações de palavras com seus sinônimos e antônimos. Também veremos algumas analogias interessantes que o algoritmo é capaz de fazer ao capturar um pouco do nosso uso da língua portuguesa.


In [3]:
from gensim.models import KeyedVectors

In [2]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 15.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy

# Carregando dados


Para esta lista nós utilizaremos vetores de palavras, também conhecidos como *embeddings*, para lingua portuguesa fornecidos pelo [NILC](http://www.nilc.icmc.usp.br/nilc/index.php). Nós utilizaremos o embedding de 50 dimensões treinado com o algoritmo Word2Vec (Continous Bag of Words) que pode ser encontrado [aqui](http://www.nilc.icmc.usp.br/embeddings) sob a licensa [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/). Para evitar problemas de mémoria utilizaremos apenas as 200 mil palavras mais comum.

In [1]:
!curl  https://raw.githubusercontent.com/alan-barzilay/NLPortugues/master/Semana%2004/data/word2vec_200k.txt --output 'word2vec_200k.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 92.0M  100 92.0M    0     0  8528k      0  0:00:11  0:00:11 --:--:-- 24.0M


In [4]:
# Carrega word2vec
model = KeyedVectors.load_word2vec_format("word2vec_200k.txt")

# Similaridade e Distância Cosseno

Como comentamos em sala de aula, podemos considerar as palavras como pontos num espaço n-dimensional e podemos examinar a proximidade delas através da similaridade cosseno:
$$s = \frac{u \cdot v}{||u|| ||v||}, \textrm{ onde } s \in [-1, 1] $$


## <font color='blue'>Questão 1 </font>
Palavras [polissemicas](https://pt.wikipedia.org/wiki/Polissemia) e [homônimas](https://pt.wikipedia.org/wiki/Hom%C3%B3nimo) são palavras que possuem mais de um significado.


Utilizando a função `model.most_similar(positive = palavra1)`, que retorna uma lista das palavras mais similares à palavra1, encontre uma palavra que possua múltiplos significados. Observe que na sua lista de 10 palavras mais similares existam palavras relacionadas a mais de um dos seus significados, lembre-se de consultar sua [documentação](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.most_similar).

Por exemplo, a palavra "manga" possui na sua lista de 10 palavras mais similares as palavras "gola" e "lapela" (que estão relacionadas ao significado de manga de uma camiseta) e a palavra "maçã" (que está relacionada ao significado da fruta manga).



In [8]:
# @title Default title text
print(len(model.key_to_index), "tokens no vocabulário")

200000 tokens no vocabulário


In [18]:
def tem_vocab(kv, w):
    return w in kv.key_to_index

palavra = "bateria"   # troque para a que você quer testar
print(tem_vocab(model, palavra))

True


In [19]:
viz = model.most_similar(positive=[palavra], topn=10)
for w, score in viz:
    print(f"{w:20s} {score:.4f}")


guitarra             0.8475
pêndula              0.8291
lambreta             0.8275
lata-velha           0.8274
corneta              0.8167
batedeira            0.8112
fieira               0.8106
buzina               0.8067
histуria             0.8062
afinação             0.8043


In [21]:
from collections import defaultdict

# Âncoras de exemplo (ajuste para seu domínio/corpora)
SENSE_ANCHORS = {
    "manga": {
        "roupa": {"gola","lapela","camisa","casaco","tecido","costura","blusa","punho"},
        "fruta": {"fruta","maçã","banana","goiaba","mamão","abacaxi","suco","pé","colheita"}
    },
    "banco": {
        "financeiro": {"agência","conta","cheque","financiamento","crédito","poupança","juros","caixa","empréstimo"},
        "assento": {"cadeira","banco_de_madeira","assento","praça","jardim","banquinho","sentar"},
        "geografia": {"areia","rio","banco_de_areia","margem"}
    },
    "sede": {
        "sede_sede": {"sede","sede_de","bebida","água","sedento"},
        "quartel_general": {"matriz","hq","escritório","filial","subsidiária","administração"}
    },
    # adicione outras palavras candidatas…
}

def checa_polissemia(kv, palavra, topn=10):
    if palavra not in kv.key_to_index:
        return {"ok": False, "motivo": "fora do vocabulário"}
    viz = [w for w,_ in kv.most_similar(positive=[palavra], topn=topn)]
    sentidos = SENSE_ANCHORS.get(palavra, {})
    hits_por_sentido = defaultdict(list)
    for sentido, anchors in sentidos.items():
        for w in viz:
            if w in anchors:
                hits_por_sentido[sentido].append(w)
    return {
        "palavra": palavra,
        "vizinhos": viz,
        "hits_por_sentido": dict(hits_por_sentido),
        "sentidos_com_hits": [s for s,hs in hits_por_sentido.items() if hs],
        "ok": sum(1 for hs in hits_por_sentido.values() if hs) >= 2
    }

print(checa_polissemia(model, "manga"))
print(checa_polissemia(model, "banco"))


{'palavra': 'manga', 'vizinhos': ['lapela', 'gola', 'cola', 'maça', 'serapilheira', 'aréola', 'cachaça', 'pantera', 'cuia', 'canela'], 'hits_por_sentido': {'roupa': ['lapela', 'gola']}, 'sentidos_com_hits': ['roupa'], 'ok': False}
{'palavra': 'banco', 'vizinhos': ['observatório', 'governo', 'consórcio', 'comitж', 'orуamento', 'tesouro', 'mercado', 'setor', 'monopólio', 'cine-theatro'], 'hits_por_sentido': {}, 'sentidos_com_hits': [], 'ok': False}


In [23]:
import numpy as np
from sklearn.cluster import KMeans

def cluster_vizinhos(model, palavra, topn=20, k=2):
    viz = model.most_similar(positive=[palavra], topn=topn)
    words = [w for w,_ in viz]
    X = np.vstack([model[w] for w in words])
    km = KMeans(n_clusters=k, n_init=10, random_state=42).fit(X)
    clusters = {}
    for w, c in zip(words, km.labels_):
        clusters.setdefault(c, []).append(w)
    return clusters

clusters = cluster_vizinhos(model, "manga", topn=20, k=2)
for cid, itens in clusters.items():
    print(f"\nCluster {cid}:")
    print(", ".join(itens))



Cluster 0:
lapela, gola, maça, aréola, cuia, sotaina, lousa, argola, alcatifa

Cluster 1:
cola, serapilheira, cachaça, pantera, canela, madeira, laranja, tequila, seda, areia, palha



**<font color='red'> Sua resposta aqui </font>**

# Sinônimos e Antônimos


As vezes é mais intuitivo trabalhar com uma medida de distancia ao invés da similaridade cosseno, para isso vamos utilizar a distancia cosseno que é simplesmente 1 - Similaridade Cosseno.

## <font color='blue'>Questão 2 </font>


Usando a função [`model.distance(palavra1,palavra2)`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.distance), encontre 3 palavras onde as palavras p1 e p2 são sinônimas e p1 e p3 são antônimas mas `distance(p1,p3)` < `distance(p1,p2)`.

Proponha uma explicação do porque esse resultado contraintuitivo acontece.






In [29]:
from gensim.models import KeyedVectors

# Carregue seu modelo já treinado (ajuste o caminho conforme seu arquivo)
# Exemplo se for word2vec binário:
# kv = KeyedVectors.load_word2vec_format("caminho/word2vec.bin", binary=True)

# Para este exemplo, vamos supor que já temos o modelo carregado em `kv`

p1 = "forte"
p2 = "robusto"     # sinônimo
p3 = "fraco"       # antônimo

# Conferir se estão no vocabulário
for w in [p1, p2, p3]:
    if w not in model.key_to_index:
        print(f"{w} não está no vocabulário")

# Calcular distâncias
d12 = model.distance(p1, p2)  # bom x excelente
d13 = model.distance(p1, p3)  # bom x ruim

print(f"Distância({p1}, {p2}) = {d12:.4f}")
print(f"Distância({p1}, {p3}) = {d13:.4f}")

if d13 < d12:
    print(f"➡ Contraintuitivo: '{p1}' está mais próximo de '{p3}' (antônimo) do que de '{p2}' (sinônimo).")


Distância(forte, robusto) = 0.3109
Distância(forte, fraco) = 0.2092
➡ Contraintuitivo: 'forte' está mais próximo de 'fraco' (antônimo) do que de 'robusto' (sinônimo).



**<font color='red'> O modelo aproxima palavras que ocorrem em contextos semelhantes, não palavras que têm o mesmo significado.
Por isso, às vezes, antônimos ficam mais próximos do que sinônimos, porque aparecem nas mesmas construções de frase, enquanto sinônimos podem variar mais no uso. </font>**

# Analogias

Existem algumas analogias famosas realizadas por vetores de palavras. O exemplo mais famoso é provavelmente "man : king :: woman : x", onde x é *queen*.

Para formular analogias vamos utilizar a função `most_similar()` que busca as palavras mais similares as listas em  `positive` e mais dissimilares as listadas em  `negative`. Para mais detalhes recomendamos consultar a sua [documentação](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.most_similar).




In [41]:
result = model.most_similar(positive=["inverno", "calor"], negative=["verão"] )

print(result)

[('vento', 0.8535684943199158), ('sedimento', 0.8110452890396118), ('resfriamento', 0.7972230315208435), ('nevoeiro', 0.7859594821929932), ('estômago', 0.7845367193222046), ('ruído', 0.7833989858627319), ('fluído', 0.7799857258796692), ('cozimento', 0.7734391689300537), ('fotoperíodo', 0.7702401876449585), ('fumo', 0.7679013609886169)]


## <font color='blue'>Questão 3 </font>
Encontre analogias que funcionam, ou seja, que a palavra esperada está no topo da lista.

Descreva sua analogia na seguinte forma:
x:y :: a:b



In [42]:
result = model.most_similar(positive=["inverno", "calor"], negative=["verão"] )

print(result)

[('vento', 0.8535684943199158), ('sedimento', 0.8110452890396118), ('resfriamento', 0.7972230315208435), ('nevoeiro', 0.7859594821929932), ('estômago', 0.7845367193222046), ('ruído', 0.7833989858627319), ('fluído', 0.7799857258796692), ('cozimento', 0.7734391689300537), ('fotoperíodo', 0.7702401876449585), ('fumo', 0.7679013609886169)]



**<font color='red'> Sua resposta aqui </font>**

## <font color='blue'>Questão 4 </font>
Encontre analogias que **Não** funcionam.

Descreva o resultado esperado da sua analogia na seguinte forma:
x:y :: a:b

E indique o valor errado de b encontrado



In [47]:

# 1ª analogia: rei : rainha :: homem : ?
result1 = model.most_similar(positive=['rainha','homem'], negative=['rei'], topn=1)
print("rei:rainha :: homem:", result1[0][0], "-> similaridade:", result1[0][1])

# 2ª analogia: rápido : devagar :: alto : ?
result2 = model.most_similar(positive=['devagar','alto'], negative=['rápido'], topn=1)
print("rápido:devagar :: alto:", result2[0][0], "-> similaridade:", result2[0][1])

# 3ª analogia: cachorro : filhote :: gato : ?
result3 = model.most_similar(positive=['filhote','gato'], negative=['cachorro'], topn=1)
print("cachorro:filhote :: gato:", result3[0][0], "-> similaridade:", result3[0][1])


# 5ª analogia: música : violão :: desenho : ?
result5 = model.most_similar(positive=['violão','desenho'], negative=['música'], topn=1)
print("música:violão :: desenho:", result5[0][0], "-> similaridade:", result5[0][1])


rei:rainha :: homem: crianção -> similaridade: 0.6878494620323181
rápido:devagar :: alto: sossegado -> similaridade: 0.7118181586265564
cachorro:filhote :: gato: frêmito -> similaridade: 0.8587124347686768
música:violão :: desenho: teclado -> similaridade: 0.8038460612297058


Mesmo que a similaridade seja alta, o modelo não garante que a resposta seja correta.

Analogias não funcionam quando o modelo não aprendeu bem a relação ou quando a relação é abstrata/menos comum.

Por isso, essas analogias são exemplos perfeitos de “analogias que não funcionam”, como a questão pede.

# Viés e preconceito adquirido

Como estes vetores são aprendidos a partir de documentos produzidos pela nossa sociedade, ele pode vir a capturar alguns preconceitos e desigualdades presentes na nossa sociedade. É importante estar ciente desse viés de nossos vetores e dos seus perigos, aplicações que utilizam esses modelos podem acabar perpetuando e até mesmo exacerbando desigualdades sociais.

Por exemplo, uma analogia problemática capturada:



In [48]:
model.most_similar(positive=['negro', 'rico'], negative=['pobre'])

[('branco', 0.663209080696106),
 ('alegre/rs', 0.6620162725448608),
 ('braga-fc', 0.6464027762413025),
 ('sporting-fc', 0.6254758238792419),
 ('côvo', 0.6254613995552063),
 ('alegre-rs', 0.6199708580970764),
 ('vermelho', 0.612277090549469),
 ('covo', 0.604120671749115),
 ('cirílicos', 0.6022458672523499),
 ('benfica-fc', 0.5965930819511414)]

Note também como diferem as palavras mais semelhantes a homem e mulher:

In [49]:
model.most_similar("homem")

[('monstro', 0.9085395932197571),
 ('bebé', 0.9072304368019104),
 ('indivíduo', 0.9050756096839905),
 ('rapaz', 0.9036115407943726),
 ('mendigo', 0.9007540345191956),
 ('rapazola', 0.8992964029312134),
 ('novelo', 0.8938027620315552),
 ('pássaro', 0.8897998929023743),
 ('cão', 0.8882535099983215),
 ('cãozinho', 0.8869855403900146)]

In [50]:
model.most_similar("mulher")

[('menina', 0.911119282245636),
 ('amiga', 0.9089193344116211),
 ('cadela', 0.9035040140151978),
 ('rapariga', 0.899989902973175),
 ('enfermeira', 0.8974366784095764),
 ('namorada', 0.8954240083694458),
 ('cafetina', 0.8932163119316101),
 ('prostituta', 0.8917951583862305),
 ('garota', 0.8906298279762268),
 ('cadelinha', 0.8902611136436462)]

## <font color='blue'>Questão 5 </font>

Utiliza a função `most_similar()` para encontrar um outro caso de viés adquirido pelos vetores e explique brevemente o tipo de viés encontrado.



In [57]:
result = model.most_similar(positive=["jovem", "tecnologia"], negative=["idoso"], topn=1)
print(result)



[('arquitetura', 0.7338955402374268)]



**<font color='red'> Sua resposta aqui </font>**

## <font color='blue'>Questão 6 </font>

Qual é a possivel origem desses vieses? Tente explicar como eles podem ter sido capturados pelos vetores de palavras.

O resultado foi "arquitetura", o que sugere que o modelo associa jovens com tecnologia e idosos não. O tipo de viés que aparece aqui é relacionado a idade e atividade/inteligência tecnológica, ou seja, o modelo reflete um estereótipo cultural: jovens usam ou dominam tecnologia mais que pessoas idosas.

Dados de treino: Modelos de word embeddings como Word2Vec são treinados em grandes corpora de texto da internet. Se o corpus tem textos que falam mais sobre "jovem" e "tecnologia" juntos (ex.: posts em redes sociais, artigos de startups), o vetor de "jovem" vai ficar mais próximo de "tecnologia".

Menor presença de "idoso" em contextos tecnológicos: Se textos associando idosos à tecnologia são raros ou negativos, o embedding de "idoso" fica distante de palavras relacionadas a tecnologia.

Co-ocorrência e contexto: Embeddings capturam co-ocorrência de palavras. Quanto mais duas palavras aparecem juntas em contextos semelhantes, mais próximos elas ficam no espaço vetorial. Por isso, estereótipos do mundo real se refletem nos vetores.